In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_essf_one_iso.py
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Makes the effective survey selection function for APOGEE using a single 
isochrone drawn from a larger grid of isochrones. Only used for mock data
generation and fitting
'''

__author__ = "James Lane"

In [ ]:
### Imports

# General
import numpy as np, pdb, sys, os, dill as pickle, multiprocessing
from tqdm.notebook import tqdm

# Set APOGEE version for the package
apogee_results_vers = 'l33'
# Year 7 is appropriate for DR16 (l33)
apo_combined_select_year = 7
os.environ['RESULTS_VERS'] = apogee_results_vers

# Astronomy
import apogee.select as apsel
import apogee.tools as apotools
from isodist import Z2FEH
import mwdust

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import iso as piso
from ges_mass import util as putil

In [ ]:
### Notebook setup

%load_ext autoreload
%autoreload 2

### Preliminaries

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','NPROCS']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,nprocs\
    = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]

# Manually set number of processors
nprocs = 36 # Go higher than the config

# Sanity
assert apotools.path._APOGEE_REDUX == apogee_results_vers,\
    'Not using correct results version!'

# Filenames
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
apogee_SF_filename = ga_dir+'apogee_SF.dat'
apogee_effSF_filename_prefix = ga_dir+'apogee_effSF_grid_inclArea'
iso_filename = ga_dir+'iso_grid.npy'

# Forcing
force_effSF = False # Force calculation of the essf even if one exists?

# Parameters for effective survey selection function
distmods = np.linspace(dmod_min, dmod_max, ndmod)
ds = 10.**(distmods/5-2)

### Loading

In [ ]:
# Selection function
print('APOGEE data release is: '+apogee_dr+', and results version is: '+apogee_results_vers)
if os.path.exists(apogee_SF_filename):
    print('Loading APOGEE selection function from '+apogee_SF_filename)
    with open(apogee_SF_filename, 'rb') as f:
        apogee_SF = pickle.load(f)
else:
    sys.exit('Could not find APOGEE selection function, make it. Exiting...')

# Dustmap and isochrones
print('Loading isochrone grid')
iso_filename = ga_dir+'iso_grid.npy'
iso = np.load(iso_filename)
print('Loading Combined19 dust map from mwdust')
dmap = mwdust.Combined19(filter='2MASS H') # dustmap from mwdust, use most recent

### Effective selection function for one isochrone
Default is to choose $Z_{ini} = 0.0010$ and $\log_{10}(\mathrm{age}) = 10$

In [ ]:
iso_zini = 0.0010
iso_logAge = 10.
print('Calculating APOGEE effective selection function for z='+str(iso_zini)+\
      ', log(age)='+str(iso_logAge))

# Isochrone elements and weights
# The [Fe/H] range is based approximately off of the locus of GE in Mg/Fe-Fe/H
iso_mask = (iso['logg'] > logg_min) &\
           (iso['logg'] < logg_max) &\
           (iso['Zini'] == iso_zini) &\
           (iso['logAge'] == iso_logAge)
H = iso['Hmag'][iso_mask]
J = iso['Jmag'][iso_mask]
K = iso['Ksmag'][iso_mask]
weights = iso['weights_imf'][iso_mask]/iso['Zini'][iso_mask]

# Create an APOGEE effective selection function object
apogee_effSF = apsel.apogeeEffectiveSelect(apogee_SF, dmap3d=dmap, MH=H, 
                                           JK0=(J-K), weights=weights)

# Function to calculate the effective selection function at one location, 
# Based on the loaded isochrone, the dust map, and including the area factor
def _calc_effSF_inclArea_one_loc(i):
    loc = apogee_SF._locations[i]
    if np.sum([np.nansum(apogee_SF._nspec_short[i]),
               np.nansum(apogee_SF._nspec_medium[i]),
               np.nansum(apogee_SF._nspec_long[i])]) < 1.:
        effSF_inclArea = np.zeros(len(ds))
    else:
        effSF_inclArea = apogee_effSF(loc, ds, MH=H, JK0=(J-K), weights=weights)\
                         *apogee_SF.area(loc)
    return effSF_inclArea

with multiprocessing.Pool(nprocs) as p:
    out = list(tqdm(p.imap(_calc_effSF_inclArea_one_loc, 
                                range(0,len(apogee_SF._locations))), 
                         total=len(apogee_SF._locations)))
apogee_effSF_grid_inclArea = np.array(out)
apogee_effSF_filename_suffix = '_z'+str(iso_zini)+'_logAge'+str(iso_logAge)+'.dat'
apogee_effSF_filename = apogee_effSF_filename_prefix+apogee_effSF_filename_suffix
with open(apogee_effSF_filename, 'wb') as f:
    pickle.dump(apogee_effSF_grid_inclArea, f)

### Effective selection function for one isochrone with no dust
Choose the same isochrone parameters, but now ignore dust, using `mwdust.Zero`

In [ ]:
dmap = mwdust.Zero(filter='2MASS H')
iso_zini = 0.0010
iso_logAge = 10.
print('Calculating APOGEE effective selection function for z='+str(iso_zini)+\
      ', log(age)='+str(iso_logAge)+' without dust')

# Isochrone elements and weights
# The [Fe/H] range is based approximately off of the locus of GE in Mg/Fe-Fe/H
iso_mask = (iso['logg'] > logg_min) &\
           (iso['logg'] < logg_max) &\
           (iso['Zini'] == iso_zini) &\
           (iso['logAge'] == iso_logAge)
H = iso['Hmag'][iso_mask]
J = iso['Jmag'][iso_mask]
K = iso['Ksmag'][iso_mask]
weights = iso['weights_imf'][iso_mask]/iso['Zini'][iso_mask]

# Create an APOGEE effective selection function object
apogee_effSF = apsel.apogeeEffectiveSelect(apogee_SF, dmap3d=dmap, MH=H, 
                                           JK0=(J-K), weights=weights)

# Function to calculate the effective selection function at one location, 
# Based on the loaded isochrone, the dust map, and including the area factor
def _calc_effSF_inclArea_one_loc(i):
    loc = apogee_SF._locations[i]
    if np.sum([np.nansum(apogee_SF._nspec_short[i]),
               np.nansum(apogee_SF._nspec_medium[i]),
               np.nansum(apogee_SF._nspec_long[i])]) < 1.:
        effSF_inclArea = np.zeros(len(ds))
    else:
        effSF_inclArea = apogee_effSF(loc, ds, MH=H, JK0=(J-K), weights=weights)\
                         *apogee_SF.area(loc)
    return effSF_inclArea

with multiprocessing.Pool(nprocs) as p:
    out = list(tqdm(p.imap(_calc_effSF_inclArea_one_loc, 
                                range(0,len(apogee_SF._locations))), 
                         total=len(apogee_SF._locations)))
apogee_effSF_grid_inclArea = np.array(out)
apogee_effSF_filename_suffix = '_z'+str(iso_zini)+'_logAge'+str(iso_logAge)+\
                               '_dmap_Zero.dat'
apogee_effSF_filename = apogee_effSF_filename_prefix+apogee_effSF_filename_suffix
with open(apogee_effSF_filename, 'wb') as f:
    pickle.dump(apogee_effSF_grid_inclArea, f)